#  A simple deep CNN
[reference](https://keras.io/examples/cifar10_cnn/)

In [99]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D, Convolution1D
import os

In [84]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
sns.set_style('whitegrid')
sns.set_palette('Set2')
# %matplotlib inline
plt.rcParams.update({'figure.titlesize': 'larger', 'legend.fontsize': 15.0})

In [85]:
batch_size = 32
num_classes = 2
epochs = 10
data_augmentation = False
# num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_trained_model.h5'

In [86]:
data =pd.read_csv('data_pre_processed.csv')

In [87]:
print('Total data shape is ', data.shape)
data.head()

,first_careunit,last_careunit,age,age_category,gender,marital_status,insurance,urea_n_min,urea_n_max,urea_n_mean,...,spo2_max_mv,spo2_mean_mv,vent_mv,rrt_mv,urineoutput_mv,oasis_mv,lods_mv,sirs_mv,thirty_days,one_year
0,1.062189,1.042783,0.020618,7.0,0.889068,0.801375,0.722122,-0.110132,0.345616,0.042146,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.338382,0.334179,-0.175699,6.0,-1.124773,0.801375,0.722122,-0.035009,1.564791,1.406861,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.062189,1.042783,-0.623562,4.0,0.889068,-2.394261,-1.738451,-0.035009,-0.296056,-0.244412,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,-0.338382,0.334179,-0.056425,7.0,0.889068,0.801375,0.722122,0.866469,-0.039387,0.257946,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,-1.038668,-0.374426,-0.659244,3.0,-1.124773,-2.394261,-1.738451,-0.260378,-0.648975,-0.588887,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
columns = list(data.columns)
X = data[columns[0:-2]].astype(np.float32) # fetures
y = data[columns[-2]].astype(np.float32)  # 30 days 
# y = data[-1].astype(np.float32)  # 

from sklearn.model_selection import train_test_split
# The data, split between train and test sets:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (38857, 110)
38857 train samples
19139 test samples


In [89]:
# # Convert class vectors to binary class matrices.
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

In [90]:
model = Sequential()
model.add(Conv1D(32, 3, padding='same', input_shape=[X_train.shape[1],1]))
model.add(Activation('relu'))
model.add(Conv1D(32, 3))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))

model.add(Conv1D(64, 3, padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(64, 3))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
# model.add(Dense(num_classes))
# model.add(Activation('softmax'))
model.add(Dense(1, activation='sigmoid'))

In [124]:
# initiate RMSprop optimizer
# opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

X_train = X_train.values[..., None].astype('float32')
X_test = X_test.values[..., None].astype('float32')
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 64, 32)            18304     
_________________________________________________________________
lstm_6 (LSTM)                (None, 64, 32)            8320      
_________________________________________________________________
lstm_7 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_35 (Dense)             (None, 1)                 33        
Total params: 34,977
Trainable params: 34,977
Non-trainable params: 0
_________________________________________________________________


In [92]:
X_train.shape

(38857, 110, 1)

In [93]:
hist = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose = 1,
          validation_data=(X_test, y_test),
          shuffle=True,
            callbacks= [keras.callbacks.EarlyStopping()] )

Not using data augmentation.
Train on 38857 samples, validate on 19139 samples
Epoch 1/10
38857/38857 [==============================] - 31s 794us/sample - loss: 0.2464 - acc: 0.9047 - val_loss: 0.1568 - val_acc: 0.9457 - los
Epoch 2/10
38857/38857 [==============================] - 31s 796us/sample - loss: 0.1642 - acc: 0.9439 - val_loss: 0.1199 - val_acc: 0.9599
Epoch 3/10
38857/38857 [==============================] - 32s 814us/sample - loss: 0.1408 - acc: 0.9534 - val_loss: 0.1072 - val_acc: 0.9641
Epoch 4/10
38857/38857 [==============================] - 30s 777us/sample - loss: 0.1297 - acc: 0.9588 - val_loss: 0.1040 - val_acc: 0.9658
Epoch 5/10
38857/38857 [==============================] - 31s 796us/sample - loss: 0.1225 - acc: 0.9614 - val_loss: 0.0967 - val_acc: 0.9694
Epoch 6/10
38857/38857 [==============================] - 30s 776us/sample - loss: 0.1145 - acc: 0.9634 - val_loss: 0.0924 - val_acc: 0.9712
Epoch 7/10
38857/38857 [==============================] - 31s 799us/s

In [94]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at C:\Users\64877\Repositories\DS590 Final Project\DS_final\Process Books\saved_models\keras_trained_model.h5 
19139/19139 [==============================] - 3s 149us/sample - loss: 0.0795 - acc: 0.9755
Test loss: 0.07950419548984967
Test accuracy: 0.97549504


# Modified VGG16
[reference](https://gist.github.com/baraldilorenzo/07d7802847aaad0a35d3)

In [109]:
model = Sequential()
model.add(ZeroPadding1D(1,input_shape=[X_train.shape[1],1]))
model.add(Convolution1D(64, 3, activation='relu'))
model.add(ZeroPadding1D(1))
model.add(Convolution1D(64, 3, activation='relu'))
model.add(MaxPooling1D(2, strides=2))

model.add(ZeroPadding1D(1))
model.add(Convolution1D(128, 3, activation='relu'))
model.add(ZeroPadding1D(1))
model.add(Convolution1D(128, 3, activation='relu'))
model.add(MaxPooling1D(2, strides=2))

model.add(ZeroPadding1D(1))
model.add(Convolution1D(256, 3, activation='relu'))
model.add(ZeroPadding1D(1))
model.add(Convolution1D(256, 3, activation='relu'))
model.add(ZeroPadding1D(1))
model.add(Convolution1D(256, 3, activation='relu'))
model.add(MaxPooling1D(2, strides=2))

model.add(ZeroPadding1D(1))
model.add(Convolution1D(512, 3, activation='relu'))
model.add(ZeroPadding1D(1))
model.add(Convolution1D(512, 3, activation='relu'))
model.add(ZeroPadding1D(1))
model.add(Convolution1D(512, 3, activation='relu'))
model.add(MaxPooling1D(2, strides=2))

model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))
model.add(Dense(1, activation='sigmoid'))



In [110]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding1d_54 (ZeroPaddi (None, 112, 1)            0         
_________________________________________________________________
conv1d_73 (Conv1D)           (None, 110, 64)           256       
_________________________________________________________________
zero_padding1d_55 (ZeroPaddi (None, 112, 64)           0         
_________________________________________________________________
conv1d_74 (Conv1D)           (None, 110, 64)           12352     
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 55, 64)            0         
_________________________________________________________________
zero_padding1d_56 (ZeroPaddi (None, 57, 64)            0         
_________________________________________________________________
conv1d_75 (Conv1D)           (None, 55, 128)           24704     
__________

In [111]:
hist = model.fit(X_train, y_train,
              batch_size=512,
              epochs=5,
              validation_data=(X_test, y_test),
              shuffle=True,
                callbacks = [keras.callbacks.EarlyStopping()])

Train on 38857 samples, validate on 19139 samples
Epoch 1/5
38857/38857 [==============================] - 245s 6ms/sample - loss: 0.4554 - acc: 0.8474 - val_loss: 0.3207 - val_acc: 0.8659
Epoch 2/5
38857/38857 [==============================] - 228s 6ms/sample - loss: 0.2552 - acc: 0.8960 - val_loss: 0.2412 - val_acc: 0.9042
Epoch 3/5
38857/38857 [==============================] - 244s 6ms/sample - loss: 0.1757 - acc: 0.9382 - val_loss: 0.1496 - val_acc: 0.9507
Epoch 4/5
38857/38857 [==============================] - 221s 6ms/sample - loss: 0.1350 - acc: 0.9562 - val_loss: 0.1315 - val_acc: 0.9583
Epoch 5/5
38857/38857 [==============================] - 226s 6ms/sample - loss: 0.1154 - acc: 0.9645 - val_loss: 0.1318 - val_acc: 0.9576


In [112]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, 'Modified_VGG.h5')
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at C:\Users\64877\Repositories\DS590 Final Project\DS_final\Process Books\saved_models\Modified_VGG.h5 
19139/19139 [==============================] - 23s 1ms/sample - loss: 0.1318 - acc: 0.9576
Test loss: 0.13183197146172113
Test accuracy: 0.9576258
